In [1]:
import os
import cv2
import numpy as np
import pylab as plt
import seaborn as sns
from itertools import islice
from operator import itemgetter

import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable

import tensorflow as tf

import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

BATCH = 8
IMG_SIZE_e = 256
IMG_SIZE_m = 224

Segmentation Models: using `keras` framework.
Num GPUs Available:  0


## Define the two models with segmentationmodels framework

In [2]:
#enviroment variable
sm.set_framework('tf.keras')

BACKBONE_e = 'efficientnetb3'
preprocess_input = get_preprocessing(BACKBONE_e)

# define model
model_e = Unet(BACKBONE_e, 
             encoder_weights='imagenet', 
             classes=1, 
             input_shape=(IMG_SIZE_e, IMG_SIZE_e, 3),
             activation='sigmoid')

model_e.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5,
                                                 beta_1=0.9, beta_2=0.999,
                                                 epsilon=1e-7,
                                                 amsgrad=False,
                                                 name='Adam'), 
              loss=sm.losses.DiceLoss() + (1 * sm.losses.BinaryFocalLoss()),
              metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)],
              run_eagerly=False
              )

In [3]:
#enviroment variable
sm.set_framework('tf.keras')

BACKBONE_m = 'mobilenetv2'
preprocess_input = get_preprocessing(BACKBONE_m)

# define model
model_m = Unet(BACKBONE_m, 
             encoder_weights='imagenet', 
             classes=1, 
             input_shape=(IMG_SIZE_m, IMG_SIZE_m, 3),
             activation='sigmoid')

model_m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5,
                                                 beta_1=0.9, beta_2=0.999,
                                                 epsilon=1e-7,
                                                 amsgrad=False,
                                                 name='Adam'), 
              loss=sm.losses.DiceLoss() + (1 * sm.losses.BinaryFocalLoss()),
              metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)],
              run_eagerly=False
              )

## Load human models weights

In [4]:
model_m.load_weights('./models/mobilenet_deepskin_human.h5')
model_e.load_weights('./models/efficientnet_deepskin_human.h5')

## Load animal models weigts

In [3]:
model_m.load_weights('./models/mobilenet_petwound_animal.h5')
model_e.load_weights('./models/efficientnet_petwound_animal.h5')

# Do whatever you want :)